In [2]:
import psycopg2
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pysal.lib import weights
from pysal.explore import esda
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import MarkerCluster


In [3]:
# Connecting the workspace to PostgreSQL parameters
hostname = 'localhost'
database = 'MGAS'
username = 'postgres'
pwd = '1234'
port_id = 5433

# Database connection
conn = psycopg2.connect(host=hostname, dbname=database, user=username, password=pwd, port=port_id)

# Define the SQL query to select data from the table
sql_query = "SELECT * FROM service_vehicle_routes;"

# Execute the query to fetch the results into a DataFrame
df = pd.read_sql_query(sql_query, conn)

# Close the database connection
conn.close()


C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
# Creating Shapely points from latitude and longitude
points = df.apply(lambda row: Point(row.longitude, row.latitude), axis=1)

# Creating a new GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=points)

# Adding CRS to GeoDataFrame
gdf.crs = {'init': 'epsg:4326'}


C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [5]:
# Load Nairobi shapefile
Nairobi = gpd.read_file('C:/Users/user/Desktop/MGAS/Nairobi.shp')

In [6]:
# Performing a DBSCAN cluster analysis to Identify spatial clusters in our data and Moran's 1 analysis to determine it's autocorrelation.
w = weights.DistanceBand.from_dataframe(gdf, threshold=0.01)
mi = esda.Moran(gdf['vehicle_id'], w)
print('Moran\'s I:', mi.I, ' p-value:', mi.p_sim)
clusterer = DBSCAN(eps=0.01, min_samples=2)
gdf['cluster'] = clusterer.fit_predict(gdf.geometry.apply(lambda x: [x.x, x.y]).tolist())


('WARNING: ', 2, ' is an island (no neighbors)')
('WARNING: ', 4, ' is an island (no neighbors)')
('WARNING: ', 6, ' is an island (no neighbors)')
('WARNING: ', 8, ' is an island (no neighbors)')
('WARNING: ', 15, ' is an island (no neighbors)')
('WARNING: ', 16, ' is an island (no neighbors)')
('WARNING: ', 18, ' is an island (no neighbors)')
('WARNING: ', 20, ' is an island (no neighbors)')
('WARNING: ', 21, ' is an island (no neighbors)')
('WARNING: ', 27, ' is an island (no neighbors)')
('WARNING: ', 30, ' is an island (no neighbors)')
('WARNING: ', 32, ' is an island (no neighbors)')
('WARNING: ', 34, ' is an island (no neighbors)')
('WARNING: ', 36, ' is an island (no neighbors)')
('WARNING: ', 38, ' is an island (no neighbors)')
('WARNING: ', 41, ' is an island (no neighbors)')
('WARNING: ', 42, ' is an island (no neighbors)')
('WARNING: ', 43, ' is an island (no neighbors)')
('WARNING: ', 47, ' is an island (no neighbors)')
('WARNING: ', 51, ' is an island (no neighbors)')
('WA

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\libpysal\weights\weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 43 disconnected components.
 There are 28 islands with ids: 2, 4, 6, 8, 15, 16, 18, 20, 21, 27, 30, 32, 34, 36, 38, 41, 42, 43, 47, 51, 53, 56, 59, 62, 63, 66, 68, 72.
  warnings.warn(message)


In [7]:
# visualizating our outputs in an interactive map using Folium library 
m = folium.Map(location=[-1.286389, 36.817223], zoom_start=12)

# Add the roads to the map
folium.GeoJson(Nairobi).add_to(m)

# Create a MarkerCluster layer for the routes
marker_cluster = MarkerCluster().add_to(m)

# Add each route as a marker to the MarkerCluster layer
for idx, row in gdf.iterrows():
    folium.Marker([row['latitude'], row['longitude']],
                  popup=f"Vehicle ID: {row['vehicle_id']}<br>Cluster: {row['cluster']}",
                  icon=folium.Icon(color='blue', icon='info-sign')).add_to(marker_cluster)

# Display the map
m

In [8]:
# Saving the map as an HTML file
m.save('C:/Users/user/Desktop/MGAS/MAPS/Part2_Task1.html')